# Segmenting and Clustering Neighborhoods in Toronto

## Question 1

In [20]:
### import pandas as pd

dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', attrs={'class': 'wikitable sortable'})

df = dfs[0]



Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [21]:
#
df = df[df.Borough != 'Not assigned']



If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [23]:

for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

print("dataframe shape: ", df.shape)

df.head(5)

dataframe shape:  (210, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
